# Setup

In [ ]:
import pandas as pd
import numpy as np
# wizualizacje
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import itertools

from sklearn.metrics import mutual_info_score, adjusted_mutual_info_score, normalized_mutual_info_score
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

# ustawienie szerokiego ekranu wyświetlacza JNotebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# opcje wyświetlania w Pandas
# - maks. 55 kolumn
# - maks. 101 wierszy
# - liczby w notacji dziesiętnej z czterema zerami po przecinku
pd.set_option('display.max_columns', 55)
pd.set_option('display.max_rows', 101)
pd.set_option('display.float_format', lambda x: f"{x:.4f}")

# opcje formatowania wykresów matplotlib
# - etykiety osi: bold
# - tekst: bold
# - domyślny rozmiar fontu=14
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['font.size'] = '14'

# Wczytanie danych

In [ ]:
# Zadanie 3
filepath = r""
df = (
    pd.read_csv(filepath, sep=';')
    
    .assign(
        **{
             k: lambda df_, col = k: pd.to_datetime(df_[col])
                for k in [
                    'Kiedy jechał*ś?'
                ]
        },
        **{ k: lambda df_, col = k: pd.to_numeric(df_[col], downcast='float')
                for k in [
                    'Koszt podróży:'
                ]
        },
        **{ k: lambda df_, col = k: pd.to_numeric(df_[col], downcast='integer')
                for k in [
                    'Liczba minut opóźnienia na starcie:', 'Liczba minut opóźnienia na mecie:'
                ]
        }
    )
    
    .astype({
        **{ k: 'string' 
               for k in [
                   'Stacja początkowa:', 'Stacja końcowa:', 'Jak wrażenia z podróży? :D (nieobowiązkowe)', 'Uwagi (nieobowiązkowe):'
               ]            
        },
        **{ k: 'category' 
               for k in [
                   'Rodzaj pociągu:', 'Pora dnia:'
               ]            
        }
    })
    
    .rename(columns={
        'Kiedy jechał*ś?'         : 'data_przejazdu',
        'Stacja początkowa:'      : 'stacja_start',
        'Stacja końcowa:'         : 'stacja_finisz',
        'Rodzaj pociągu:'         : 'rodzaj_pociagu',
        'Uwagi (nieobowiązkowe):' : 'uwagi',
        'Koszt podróży:'          : 'cena_biletu',
        'Pora dnia:'              : 'pora_dnia_przejazdu',
        'Liczba minut opóźnienia na starcie:' : 'opoznienie_start',
        'Liczba minut opóźnienia na mecie:'   : 'opoznienie_finisz',
        'Jak wrażenia z podróży? :D (nieobowiązkowe)' : 'wrazenia'
    })
    
    .drop(columns=['wrazenia', 'uwagi'])
    .drop(index=[30, 50])
    
    .assign(
        pora_dnia_przejazdu = lambda df_: np.select(
        [
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_start.str.contains('Legionowo')).astype(bool),
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_finisz.str.contains('Legionowo')).astype(bool),
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_finisz.str.contains('Gdask')).astype(bool)
        ],
        [
            'rano',
            'popołudnie',
            'popołudnie'            
        ],
        df_.pora_dnia_przejazdu
    ))
    
    .reset_index(drop=True)
    
    .assign(
        czy_szkoda = lambda df_: np.select(
            [
                df_.opoznienie_finisz.gt(10) & df_.opoznienie_finisz.le(40),
                df_.opoznienie_finisz.gt(40)
            ],
            [
                1,
                2
            ],
            0
        ),
        
        wyplata_szkoda = lambda df_: np.where(
            df_.opoznienie_finisz.gt(10),
            df_.cena_biletu * np.exp(df_.opoznienie_finisz.div(100)) * df_.czy_szkoda,
            0.0
        )
    )
    
    .sort_values(by='data_przejazdu')
    .reset_index(drop=True)
)
df

# Spis treści
1. [Feature engineering](#fe)
2. [Informacja wzajemna](#mutual)

***
***

# <a id='fe'>Feature engineering</a> 
1. Jakie zmiany w danych byś przeprowadził na podstawie swojej analizy eksploracyjnej? Zastosuj je rozszerzając swój łańcuch 'method chaining'
2. Prześledź ich związek ze zmiennymi wyjaśnianymi

In [ ]:
# Zadanie 2


In [ ]:
# Zadanie 2


# <a id='mutual'>Informacja wzajemna</a> 
1. Policz wskaźniki informacji wzajemnej wszystkich (każda z każdą) zmiennych poprzez: 
    1. mutual_info_score, 
    2. adjusted_mutual_info_score, 
    3. normalized_mutual_info_score
2. Policz wskaźniki informacji wzajemej wszystkich (każda z każdą) zmiennych poprzez użycie mutual_info_classif oraz mutual_info_regression


In [ ]:
# Zadanie 1A



In [ ]:
# Zadanie 1B


In [ ]:
# Zadanie 1C

In [ ]:
# Zadanie 2
# Uwaga - wykonanie zadania wymaga, aby zakodować zmienne kategorialne jako numeryczne
# Kod poniżej dokładnie to wykonuje - dlaczego tak? będzie na przyszłych zajęciach

column_trans = ColumnTransformer(
    [
        (
            "onehot_categorical",
            OrdinalEncoder(), [
#                 <tu wpisz listę swoich zmiennych kategorialnych>
            ],
        ),
        (
            "passthrough",
            'passthrough', [
#                 <tu wpisz listę swoich zmiennych ciągłych>
            ]
        )
    ],
    verbose=True
)

X_coded = column_trans.fit_transform(df)
transformer_feature_names = []
for name, transformer, features in column_trans.transformers_:
    if hasattr(transformer, 'get_feature_names'):
        transformer_feature_names.extend(
            [f"{name}_{f}" for f in transformer.get_feature_names(features)]
        )
    else:
        transformer_feature_names.extend(features)
df_coded = pd.DataFrame(X_coded, columns=transformer_feature_names)        
